# Assignment 1

In [148]:
import re
import helper
import numpy as np
import pandas as pd

ODI_DATASET_PATH = './data/ODI-2022.csv'

In [149]:
# # load csv file to pandas dataframe to retrieve column names
# df_temp= pd.read_csv("./data/ODI-2022.csv", delimiter=";")
# df_temp

In [151]:
# Initialize 2 dimensional array
dataset = helper.get_data(ODI_DATASET_PATH)
# Initialize columns
columns = list(zip(*dataset))

# print("What programme are you in? = {}".format(columns[1]))
# print("Have you taken a course on machine learning? = {}".format(columns[2]))
# print("Have you taken a course on information retrieval?; = {}".format(columns[3]))
# # mu is yes, sigma is no
# print("Have you taken a course on statistics? = {}".format(columns[4]))
# print("Have you taken a course on databases? = {}".format(columns[5]))
# print("What is your gender? = {}".format(columns[6]))
# print("Chocolate makes you..... = {}".format(columns[7]))
# print("When is your birthday (date)? = {}".format(columns[8]))
# print("Number of neighbors sitting around you? = {}".format(columns[9]))
# print("Did you stand up? = {}".format(columns[10]))
# print("What is your stress level (0-100)? = {}".format(columns[11]))
# print("You can get 100 euros if you win a local DM competition, or we don’t hold any competitions and I give everyone some money (not the same amount!). How much do you think you would deserve then? = {}".format(columns[12]))
# print("Give a random number = {}".format(columns[13]))
# print("Time you went to be Yesterday = {}".format(columns[14]))
# print("What makes a good day for you (1)?; = {}".format(columns[15]))
# print("What makes a good day for you (2)? = {}".format(columns[16]))



In [165]:
# load csv file to pandas dataframe to retrieve column names
df_temp= pd.read_csv("./data/ODI-2022.csv", delimiter=";")
column_names = df_temp.columns

# convert dataset to pandas dataframe using column names from df_temp
df = pd.DataFrame(dataset)
df.columns=column_names

df_temp = df_temp.drop(labels=148, axis=0)
df_temp = df_temp.drop(labels=205, axis=0)

## clean all column values
# remove unimportant columns
df = df.drop("Tijdstempel", axis=1) # remove Tijdstempel column because it has no meaning
df = df.drop("Did you stand up?", axis=1) # remove stand up column because it has no meaning
df = df.drop("Number of neighbors sitting around you?", axis=1) # remove neighbours column because it has no meaning
df = df.drop("Give a random number", axis=1) # remove random number column because it has no meaning

# convert column values to boolean for yes/no questions
df['Have you taken a course on machine learning?'] = df['Have you taken a course on machine learning?'].replace(['yes','no','unknown'],[True,False,False])
df['Have you taken a course on information retrieval?'] = df['Have you taken a course on information retrieval?'].replace([1,0,-1],[True,False, False])
df['Have you taken a course on statistics?'] = df['Have you taken a course on statistics?'].replace(['mu','sigma', 'unknown'],[True,False,False])
df['Have you taken a course on databases?'] = df['Have you taken a course on databases?'].replace(['ja','nee', 'unknown'],[True,False,False])

# only keep years in birthday column
def get_year(date):
    year = re.search('[0-9]{4}', date)
    symbols = {'/', '.', '-'}
    if year == None:
        if len(date) > 4 and any(sym in date for sym in symbols):
            year = "19" + date[-2:]
            if year.isdecimal():
                year = int(year)
                if 1930 > year:
                    return np.nan
                elif year > 2005:
                    return np.nan
                else:
                    return year
            else:
                return np.nan
        else:
            return np.nan
    else:
        year = int(year.group(0))
        if 1930 > year:
            return np.nan
        elif year > 2005:
            return np.nan
        else:
            return year

# iterate all rows of birthday column and set all birthdays to year
for index, row in df.iterrows():
    birthday = (row["When is your birthday (date)?"])
    new_birthday = get_year(birthday)
    df['When is your birthday (date)?'] = df['When is your birthday (date)?'].replace([birthday],new_birthday)

# clean stress level column --> set all values < 0 to 0 and > 100 to 100
df = df.astype({'What is your stress level (0-100)?': int}, errors='raise')
df['What is your stress level (0-100)?'].values[df['What is your stress level (0-100)?'].values > 100] = 100
df['What is your stress level (0-100)?'].values[df['What is your stress level (0-100)?'].values < 0] = 0

# clean competition column --> set all values < 0 to 0 and > 100 to 100
df.rename(columns = {'You can get 100 euros if you win a local DM competition, or we don’t hold any competitions and I give everyone some money (not the same amount!). How much do you think you would deserve then? ':'competition'}, inplace = True)
df = df.astype({'competition': int}, errors='raise')
df['competition'].values[df['competition'].values > 100] = 100
df['competition'].values[df['competition'].values < 0] = 0

# clean bedtime column
df.rename(columns={'Time you went to be Yesterday': 'bedtime'}, inplace=True)

# for new, old in zip(list(df_temp['Time you went to be Yesterday']), list(df['bedtime'])):
#     df['bedtime'] = df['bedtime'].replace([old], new)

# for new, old in zip(list(df_temp['Time you went to be Yesterday']), list(df['bedtime'])):
#     print(new, old)

def clean_time(old):
    print(old)
    pass

for index, row in df.iterrows():
    old_time = (row['bedtime'])
    print(old_time)
    # df['bedtime'] = df['bedtime'].replace([old_time], new_time)


# What kinds of attributes are there
## Boolean, object, integer, float
# print(df['What is your gender?'].unique())
# print(df.dtypes)
# display(df)


# Ranges of values
# Distribution of values,
# relationships between attributes
# and so on.

23:00
32
x
14:00
07:20
2130
00:35
23
23:00
12 hours
Banana
00:00
to be?
00:00
12:00
4 years ago
noon
12:55
12
12
20:00
definitely 7
03:00
Ja
2.30AM
Yellow
23:32
?
12
23.30
12:30
90
23
5
5
What?
?
01.30
1:34
09:00
midnight
huh
Yesterday
11
23:30
1
23:00
0200
Today
Haha
4
01:00
10:00 pm
23:45
24
12.00
Yes
N/A
00:34
13:15
7
Gym
2.00
23:00
22:30
15:30
12
yes
11pm
2 am
2300
0530 am
12
13:12
01.00
4 o' clock
4:35
12
12
2:00
12:00
Wot
10:45
20.09
00:00
10
22:00
24
12
12:45
6 am
23
7 hours
12
12.15
Ω
9.00 am
I do not know
10:00
00:45
11.30pm
6
23:00
10:21:39
4:20
idk
12
00:55
23:30
12
today
0100
23:30
01:30
21:20
2am
45
3AM
23:30
01:30
Dizzy Miss Lizzy was a terrible song to end Help! with
02:20
1.00
23:00
23:40
05:00
1am
00
20.30
01.30
23:00
23:00
2:30
11:30
22:00
11:45
00:40
00:15
7:00
00:07
23pm
4:15
4am
01:53
0:00
1 am
01.00
00:00
01:00
23:15
01:00
00.20
01:00
0:15h
23:00
23:00
4.30
23.30
23:00
12 am
23
03:00
2230
1am
06:00
21:43
23:45
01:00
23.45
00:12
23:00
23:00
2:00am
12:00
23:00
9 AM


In [6]:
# How many records are there
n_records = df.shape[0]
print(f"There are {n_records} records in the file.")

# How many attributes?
n_attributes = len(df.columns)
print(f"There are {n_attributes} attributes in the file.")

# Make various plots of the data. Is there something interesting worth reporting? Re-
# port the figures, discuss what is in them. What meaning do those bars, lines, dots, etc.
# convey? Please select essential and interesting plots for discussion, as you have limited
# space for reporting your findings (see details in a later section).